In [ ]:
import pandas as pd
from google.colab import files

# Function to calculate AQI
def calculate_aqi(concentration, breakpoints):
    for bp in breakpoints:
        C_low, C_high, AQI_low, AQI_high = bp
        if C_low <= concentration <= C_high:
            aqi = (concentration - C_low) / (C_high - C_low) * (AQI_high - AQI_low) + AQI_low
            return aqi
    return None  # In case the concentration is out of bounds

# Define breakpoints for each pollutant
breakpoints = {
    'CO': [(0.0, 4.4, 0, 50), (4.5, 9.4, 51, 100), (9.5, 12.4, 101, 150),
           (12.5, 15.4, 151, 200), (15.5, 30.4, 201, 300), (30.5, float('inf'), 301, 500)],
    'NO2': [(0.0, 0.053, 0, 50), (0.054, 0.100, 51, 100), (0.101, 0.200, 101, 150),
            (0.201, 0.400, 151, 200), (0.401, 0.600, 201, 300), (0.601, float('inf'), 301, 500)],
    'Ozone': [(0.0, 0.054, 0, 50), (0.055, 0.070, 51, 100), (0.071, 0.085, 101, 150),
              (0.086, 0.105, 151, 200), (0.106, 0.200, 201, 300), (0.201, float('inf'), 301, 500)],
    'SO2': [(0.0, 0.035, 0, 50), (0.036, 0.075, 51, 100), (0.076, 0.185, 101, 150),
            (0.186, 0.304, 151, 200), (0.305, 0.604, 201, 300), (0.605, float('inf'), 301, 500)],
}

# Define air quality levels based on AQI ranges
def get_aqi_level(aqi):
    if aqi is None:
        return "N/A"
    elif aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Moderate"
    elif aqi <= 150:
        return "Unhealthy for sensitive groups"
    elif aqi <= 200:
        return "Unhealthy"
    elif aqi <= 300:
        return "Very Unhealthy"
    else:
        return "Hazardous"

# Read CSV file (adjust the filename as needed)
uploaded = files.upload()  # Upload the file and read it
df = pd.read_csv(next(iter(uploaded)))

# Calculate AQI for each row
aqi_results = []

for index, row in df.iterrows():
    co_aqi = calculate_aqi(row['CO'], breakpoints['CO'])
    no2_aqi = calculate_aqi(row['NO2'] / 1000, breakpoints['NO2'])  # Convert ppb to ppm
    ozone_aqi = calculate_aqi(row['Ozone'] / 1000, breakpoints['Ozone'])  # Convert ppb to ppm
    so2_aqi = calculate_aqi(row['SO2'] / 1000, breakpoints['SO2'])  # Convert ppb to ppm

    # Handle None values by setting them to a low number for max calculation
    overall_aqi = max(co_aqi if co_aqi is not None else -1,
                      no2_aqi if no2_aqi is not None else -1,
                      ozone_aqi if ozone_aqi is not None else -1,
                      so2_aqi if so2_aqi is not None else -1)

    # Only append valid AQI values (non-negative)
    aqi_results.append(overall_aqi if overall_aqi != -1 else None)

# Add results to DataFrame with distinct column names
df['AQI'] = aqi_results
df['AQI_Level'] = df['AQI'].apply(get_aqi_level)  # Renamed to 'AQI_Level'

# Save the results to a new CSV file
output_filename = 'aqi_results.csv'
df.to_csv(output_filename, index=False)

# Download the new CSV file
files.download(output_filename)



Saving aqi_levels.csv to aqi_levels.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>